In [1]:
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from ydata_profiling import ProfileReport
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
import plotly.express as px
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.ensemble import RandomForestClassifier
from warnings import filterwarnings
filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from itertools import combinations
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.metrics import silhouette_score
from sklearn.cluster import AgglomerativeClustering
from factor_analyzer import FactorAnalyzer
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import r2_score
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFECV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import silhouette_samples
from collections import Counter
from kmodes.kprototypes import KPrototypes
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.cluster import DBSCAN

# Without PCA 

In [2]:
data = pd.read_csv('data (3).csv', index_col='date')
def Basic_Preprocessing(df):
    df.index = pd.to_datetime(df.index)
    df.index = df.index.date  

    
    df.drop(columns=['country','street'],inplace=True)
    df = df[df['price'] != 0]
    df['yr_renovated'] = df['yr_renovated'].apply(lambda x: 1 if x!=0 else 0)
    df['waterfront'] =  df['waterfront'].apply(lambda x: 1 if x!=0 else 0)
    df['yr_built'] = df['yr_built'].apply(lambda x: 1 if x > df['yr_built'].median() else 0)
    df['sqft_total'] = df['sqft_basement'] + df['sqft_above']
    df.drop(columns=['sqft_basement','sqft_above'],inplace=True)
    #df.drop(columns=['statezip'],inplace=True)
    df['statezip'] = df['statezip'].apply(lambda x: int(str(x).split()[1]))
    df['month'] = pd.to_datetime(df.index).month
    df['date'] = pd.to_datetime(df.index).day
    
    df.reset_index(drop=True, inplace=True)

    return df 

df = Basic_Preprocessing(data)

X = df.drop(columns=['price'])
Y= df['price']



In [3]:
X_train , X_test , y_train , y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

In [4]:

###########  One hot encoding 
combined_df = pd.concat([X_train, X_test])
dummy_cols = ['city']
combined_dummy_df = pd.get_dummies(combined_df[dummy_cols], drop_first=True)
X_train_dummy = combined_dummy_df.iloc[:len(X_train)]
X_test_dummy = combined_dummy_df.iloc[len(X_train):]
X_train_dummy = X_train_dummy.astype(int)
X_test_dummy = X_test_dummy.astype(int)
X_train = X_train.drop(columns=dummy_cols)
X_test = X_test.drop(columns=dummy_cols)
X_train = pd.concat([X_train, X_train_dummy], axis=1)
X_test = pd.concat([X_test, X_test_dummy], axis=1)



In [5]:
from sklearn.ensemble import IsolationForest
clf = IsolationForest(contamination=0.1)  
clf.fit(X_train)
y_pred = clf.predict(X_train)


In [6]:
X_train['cluster_labels'] = y_pred
X_train_filtered = X_train[X_train['cluster_labels'] == 1]
indexes_1 = X_train_filtered.index
y_train_filtered = y_train.loc[indexes_1]


In [7]:
vars_to_transform = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'view', 'condition', 'sqft_total']

In [8]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform on training data
X_train_filtered =X_train_filtered.copy()  # Create a copy to avoid modifying the original data
X_train_filtered[vars_to_transform] = scaler.fit_transform(X_train_filtered[vars_to_transform])

# Transform test data using the scaler fitted on training data
X_test =X_test.copy()  # Create a copy to avoid modifying the original data
X_test[vars_to_transform] = scaler.transform(X_test[vars_to_transform])

In [9]:
y_train_filtered = np.log1p(y_train_filtered)
y_test = np.log1p(y_test)

In [10]:
gb_params = {
    'n_estimators': [50, 100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'subsample': [0.5, 0.75, 1.0],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Create GradientBoostingRegressor
gb_reg = GradientBoostingRegressor()

# Grid search
grid_search = GridSearchCV(estimator=gb_reg, param_grid=gb_params,
                           cv=5, scoring='neg_mean_squared_error',
                           n_jobs=-1)
# Fit GridSearchCV
grid_search.fit(X_train_filtered, y_train_filtered)

# Get the best model
best_gb_model_1 = grid_search.best_estimator_



In [14]:
from sklearn.metrics import r2_score
best_gb_model_1.fit(X_train_filtered, y_train_filtered)
y_train_pred = best_gb_model_1.predict(X_train_filtered)
r_squared = r2_score(y_train_filtered, y_train_pred)
n = len(y_train)
p =X_train_filtered.shape[1]
adjusted_r_squared = 1 - (1 - r_squared) * (n - 1) / (n - p - 1)
print("Adjusted R-squared:", adjusted_r_squared)

########## calculate the adj R squred value for testing ##########
best_gb_model_1.fit(X_test, y_test)
y_test_pred = best_gb_model_1.predict(X_test)
r_squared_test = r2_score(y_test, y_test_pred)
n_test = len(y_test)
p_test = X_test.shape[1]
adjusted_r_squared_test = 1 - (1 - r_squared_test) * (n_test - 1) / (n_test - p_test - 1)
print("Adjusted R-squared (Test):", adjusted_r_squared_test)

Adjusted R-squared: 0.8867311082309014
Adjusted R-squared (Test): 0.9598394873994065


In [16]:
rmse_train = np.sqrt(mean_squared_error(y_train_filtered, y_train_pred))
print("RMSE (Training):", rmse_train)
# Calculate RMSE for testing set
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
print("RMSE (Testing):", rmse_test)

RMSE (Training): 0.166732687789927
RMSE (Testing): 0.10501003588758821


In [17]:
feature_importances = best_gb_model_1.feature_importances_
feature_names = X_train_filtered.columns
feature_importance_dict = dict(zip(feature_names, feature_importances))
sorted_feature_importance = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)
print("Feature Importances:")
for feature, importance in sorted_feature_importance:
    print(f"{feature}: {importance}")

Feature Importances:
sqft_total: 0.20356602013367198
sqft_living: 0.13108069516698892
bathrooms: 0.08325868433925443
statezip: 0.06951950729654968
sqft_lot: 0.05914294821623591
bedrooms: 0.05663561155192297
view: 0.043805743353218435
city_Kent: 0.0283432338593558
city_Bellevue: 0.027838473451168674
city_Auburn: 0.024049301637542275
city_Mercer Island: 0.023657233727055058
date: 0.02184247963455702
city_Seattle: 0.020940424481868093
city_Federal Way: 0.01823869475642739
city_Tukwila: 0.017496622813805752
city_Renton: 0.01727473402355781
city_Covington: 0.01577379127951426
city_Yarrow Point: 0.015569705399316236
floors: 0.014360791426782147
city_Redmond: 0.013936092750795452
city_Des Moines: 0.013925013800256522
waterfront: 0.011784449763335675
city_Clyde Hill: 0.009322311292936836
condition: 0.007694626707719203
city_Maple Valley: 0.007650271507744456
yr_built: 0.007150260775733409
city_Kirkland: 0.004992967611725976
month: 0.004423990154238015
yr_renovated: 0.003763825754636304
city_Sa

In [ ]:
# Calculate MAPE for training set
mape_train = np.mean(np.abs((y_train_filtered - y_train_pred) / y_train_filtered)) * 100
print("Mean Absolute Percentage Error (Training):", mape_train)

# Calculate MAPE for testing set
mape_test = np.mean(np.abs((y_test - y_test_pred) / y_test)) * 100
print("Mean Absolute Percentage Error (Testing):", mape_test)

In [ ]:
grid_search.best_params_

In [ ]:
feature_importances = best_gb_model_1.feature_importances_
feature_names = X_train_filtered.columns
feature_importance_dict = dict(zip(feature_names, feature_importances))
sorted_feature_importance = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)


print("Feature Importances:")
for feature, importance in sorted_feature_importance:
    print(f"{feature}: {importance}")

In [15]:
# Define the parameter grid for Random Forest
rf_params = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [3, 5, 7, 9],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Create RandomForestRegressor
rf_reg = RandomForestRegressor()

# Grid search
rf_grid_search = GridSearchCV(estimator=rf_reg, param_grid=rf_params,
                              cv=5, scoring='neg_mean_squared_error',
                              n_jobs=-1)

# Fit GridSearchCV
rf_grid_search.fit(X_train_filtered, y_train_filtered)

# Get the best Random Forest model
best_rf_model = rf_grid_search.best_estimator_

##########################################################################################################
###########################################################################################################
##########   calculate  the adj R squred value   for training 
from sklearn.metrics import r2_score
best_rf_model.fit(X_train_filtered, y_train_filtered)
y_train_pred_rf = best_rf_model.predict(X_train_filtered)
r_squared_rf = r2_score(y_train_filtered, y_train_pred_rf)
n_train = len(y_train_filtered)
p_train = X_train_filtered.shape[1]
adjusted_r_squared_rf = 1 - (1 - r_squared_rf) * (n_train - 1) / (n_train - p_train - 1)
print("Adjusted R-squared (Random Forest - Training):", adjusted_r_squared_rf)

##########   calculate  the adj R squred value   for testing ##########y_test_pred = best_gb_model_1.predict(x_test_cluster_1)
best_rf_model.fit(X_test, y_test)
y_test_pred_rf = best_rf_model.predict(X_test)
r_squared_test_rf = r2_score(y_test, y_test_pred_rf)
n_test = len(y_test)
p_test = X_test.shape[1]
adjusted_r_squared_test_rf = 1 - (1 - r_squared_test_rf) * (n_test - 1) / (n_test - p_test - 1)
print("Adjusted R-squared (Random Forest - Testing):", adjusted_r_squared_test_rf)

######################################################################################
# Calculate RMSE for training set
rmse_train_rf = np.sqrt(mean_squared_error(y_train_filtered, y_train_pred_rf))
print("RMSE (Random Forest - Training):", rmse_train_rf)

# Calculate RMSE for testing set
rmse_test_rf = np.sqrt(mean_squared_error(y_test, y_test_pred_rf))
print("RMSE (Random Forest - Testing):", rmse_test_rf)

Adjusted R-squared (Random Forest - Training): 0.6918285662792178
Adjusted R-squared (Random Forest - Testing): 0.7497954129057374
RMSE (Random Forest - Training): 0.27498718237786884
RMSE (Random Forest - Testing): 0.26210711907325335


In [ ]:
# Calculate MAPE for training set
mape_train = np.mean(np.abs((y_train_filtered - y_train_pred) / y_train_filtered)) * 100
print("Mean Absolute Percentage Error (Training):", mape_train)

# Calculate MAPE for testing set
mape_test = np.mean(np.abs((y_test - y_test_pred) / y_test)) * 100
print("Mean Absolute Percentage Error (Testing):", mape_test)

In [21]:
rf_grid_search.best_params_

{'max_depth': 9,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 300}

In [22]:
# Calculate MAPE for training set
mape_train_rf = np.mean(np.abs((y_train_filtered - y_train_pred_rf) / y_train_filtered)) * 100
print("Mean Absolute Percentage Error (Random Forest - Training):", mape_train_rf)

# Calculate MAPE for testing set
mape_test_rf = np.mean(np.abs((y_test - y_test_pred_rf) / y_test)) * 100
print("Mean Absolute Percentage Error (Random Forest - Testing):", mape_test_rf)

Mean Absolute Percentage Error (Random Forest - Training): 1.5761137518777706
Mean Absolute Percentage Error (Random Forest - Testing): 1.5691654296427504
